In [5]:
import tensorflow as tf
from tensorflow.python.framework import ops

ops.reset_default_graph()

### 自定义Esitimator

In [6]:
TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [11]:
def maybe_download():
    train_path = tf.keras.utils.get_file(TRAIN_URL.split('/')[-1], TRAIN_URL,cache_dir="./data")
    test_path = tf.keras.utils.get_file(TEST_URL.split('/')[-1], TEST_URL,cache_dir="./data")

    return train_path, test_path

In [14]:
import pandas as pd

In [16]:
def load_data(y_name ='Species' ):
    train_path,test_path = maybe_download()
    train = pd.read_csv(train_path,names=CSV_COLUMN_NAMES,header=0)
    train_x,train_y = train,train.pop(y_name)
              
    test = pd.read_csv(test_path,names=CSV_COLUMN_NAMES,header=0)
    test_x ,test_y= test,test.pop(y_name)
    return (train_x,train_y),(test_x,test_y)

In [34]:
def train_input_fn(features,labels,batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features),labels))
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)
    
    return dataset.make_one_shot_iterator().get_next()

In [40]:
def eval_input_fn(features,labels,batch_size):
    features = dict(features)
    if labels is None:
        inputs = features
    else:
        inputs = (features,labels)
    
    dataset = tf.data.Dataset.from_tensor_slices(inputs)
    dataset = dataset.batch(batch_size)
    return dataset.make_one_shot_iterator().get_next()

In [21]:
(train_x,train_y),(test_x,test_y) = load_data()

#### 定义feature columns

In [22]:
my_features_columns = []
for key in train_x.keys():
    my_features_columns.append(tf.feature_column.numeric_column(key=key))
print(my_features_columns)

[_NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


#### 定义模型

In [36]:
def model_fn(features,labels,mode,params):
    # 创建输入层 第一个参数为特征，第二参数为 特征的key
    net = tf.feature_column.input_layer(features,params['feature_columns'])
    # 创建神经网络
    for units in params['hidden_units']:
        net = tf.layers.dense(net,units=units,activation=tf.nn.relu)
    
    # 创建输出层
    logits = tf.layers.dense(inputs=net,units=params['n_classes'],activation=None)
    
    predicted_classes = tf.argmax(logits,axis=1)
    
    # 预测 In this case, the model function must return a tf.estimator.EstimatorSpec containing the prediction.
    # The predictions holds the following three key/value pairs:

    #   class_ids   holds the class id (0, 1, or 2) representing the model's prediction of the most likely species for this example.
    #   probabilities   holds the three probabilities (in this example, 0.02, 0.95, and 0.03)
    #   logit   holds the raw logit values (in this example, -1.3, 2.6, and -0.9)
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions ={
            'class_ids':predicted_classes[:,tf.newaxis],
            'probabilities':tf.nn.softmax(logits),
            'logit':logits
        }
        return tf.estimator.EstimatorSpec(mode,predictions)
    
    loss = tf.losses.sparse_softmax_cross_entropy(logits=logits,labels=labels)
    
    # 估计 In this case, the model function must return a tf.estimator.EstimatorSpec containing the model's loss and optionally one or more metrics.
    if mode == tf.estimator.ModeKeys.EVAL:
        accuracy = tf.metrics.accuracy(labels=labels,predictions=predicted_classes,name="eval_op")
        metric={'accuracy':accuracy}
        # 生成标量
        tf.summary.scalar('accuracy',accuracy[1])
        return tf.estimator.EstimatorSpec(mode,loss=loss,eval_metric_ops=metric)
    
    # 训练In this case, the model function must return an EstimatorSpec that contains the loss and a training operation.
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdagradOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(loss,global_step = tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode,loss = loss,train_op = train_op)

#### 定义Estimator

In [37]:
config = tf.estimator.RunConfig(model_dir="./model_custom")
risi_classes = tf.estimator.Estimator(model_fn=model_fn,params={'feature_columns':my_features_columns,'hidden_units':[10,10],'n_classes':3},config=config)

INFO:tensorflow:Using config: {'_model_dir': './model_custom', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000153AB2A2710>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [43]:
risi_classes.train(input_fn=lambda:train_input_fn(train_x,train_y,100),steps=10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model_custom\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1000 into ./model_custom\model.ckpt.
INFO:tensorflow:loss = 0.81500155, step = 1000
INFO:tensorflow:global_step/sec: 435.521
INFO:tensorflow:loss = 0.8084918, step = 1100 (0.231 sec)
INFO:tensorflow:global_step/sec: 600.808
INFO:tensorflow:loss = 0.78091955, step = 1200 (0.166 sec)
INFO:tensorflow:global_step/sec: 608.136
INFO:tensorflow:loss = 0.75943506, step = 1300 (0.164 sec)
INFO:tensorflow:global_step/sec: 615.644
INFO:tensorflow:loss = 0.7807152, step = 1400 (0.161 sec)
INFO:tensorflow:global_step/sec: 615.643
INFO:tensorflow:loss = 0.72056764, step = 1500 (0.163 sec)
INFO:tensorflow:global_step/sec: 611.865
INFO:tensorflow:loss = 0.732

INFO:tensorflow:loss = 0.36364818, step = 8900 (0.174 sec)
INFO:tensorflow:global_step/sec: 453.337
INFO:tensorflow:loss = 0.36634204, step = 9000 (0.220 sec)
INFO:tensorflow:global_step/sec: 403.782
INFO:tensorflow:loss = 0.34842965, step = 9100 (0.249 sec)
INFO:tensorflow:global_step/sec: 457.496
INFO:tensorflow:loss = 0.36036336, step = 9200 (0.219 sec)
INFO:tensorflow:global_step/sec: 530.501
INFO:tensorflow:loss = 0.3360127, step = 9300 (0.189 sec)
INFO:tensorflow:global_step/sec: 443.262
INFO:tensorflow:loss = 0.32671094, step = 9400 (0.226 sec)
INFO:tensorflow:global_step/sec: 519.45
INFO:tensorflow:loss = 0.34393716, step = 9500 (0.192 sec)
INFO:tensorflow:global_step/sec: 566.671
INFO:tensorflow:loss = 0.34505722, step = 9600 (0.177 sec)
INFO:tensorflow:global_step/sec: 586.673
INFO:tensorflow:loss = 0.3315487, step = 9700 (0.170 sec)
INFO:tensorflow:global_step/sec: 566.671
INFO:tensorflow:loss = 0.34463927, step = 9800 (0.176 sec)
INFO:tensorflow:global_step/sec: 551.016
INF

In [44]:
eval_result = risi_classes.evaluate(input_fn=lambda:eval_input_fn(test_x,test_y,100))
print(eval_result)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-07-07:54:34
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model_custom\model.ckpt-11000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-07-07:54:35
INFO:tensorflow:Saving dict for global step 11000: accuracy = 0.93333334, global_step = 11000, loss = 0.3808279
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 11000: ./model_custom\model.ckpt-11000
{'accuracy': 0.93333334, 'loss': 0.3808279, 'global_step': 11000}


In [45]:
# Generate predictions from the model
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

In [51]:
predictions = risi_classes.predict(input_fn=lambda:eval_input_fn(features=predict_x,labels=None,batch_size=100))

for pred_dict,expec in zip(predictions,expected):
    template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]
    
    print(template.format(iris_data.SPECIES[class_id],
                              100 * probability, expec))
    

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model_custom\model.ckpt-11000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

Prediction is "Setosa" (85.8%), expected "Setosa"

Prediction is "Versicolor" (53.3%), expected "Versicolor"

Prediction is "Virginica" (69.4%), expected "Virginica"
